In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

df_chamado = spark.read.table("v_credit.silver.tb_chamado")
df_pesquisa = spark.read.table("v_credit.silver.tb_pesquisa")

TABELA_DESTINO = "v_credit.gold.ft_performance_agente"

In [0]:
df_gold = (
    df_chamado.alias("c")
    .join(df_pesquisa.alias("p"), on="cd_chamado", how="left")
    .filter(F.col("c.cd_atendente").isNotNull())
    .groupBy(
        F.col("c.cd_atendente")
    )
    .agg(
        F.count("*").alias("qtd_chamados_atendidos"),
        F.sum("c.tm_duracao").alias("val_tempo_trabalhado_seg"),
        F.avg("p.nu_nota").alias("val_nota_media")
    )
    .withColumn("pk_performance", F.md5(F.concat(
        F.col("cd_atendente").cast("string")
    )))
)

DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.pk_performance = s.pk_performance"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Fato Performance Agente carregada (Agregada por Dia)!")